In [1]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact

## Example 1 and 2: Finding the area of a triange under the tangent of $ \frac {1}{x}$

In [2]:
#define function

def f(x):
    return 1 / x

def f_der(x):
    return (-1) / (x ** 2) 

def tangent_y_intercept(x):
    return f(x) - f_der(x) * x

def tangent_x_intercept(x):
    return ( f_der(x) * x - f(x) ) / f_der(x)

def area_under_line(x):
    return 0.5 * tangent_y_intercept(x) * tangent_x_intercept(x)


In [3]:
def plot_line(m, c, x, ax):
    y = m * x + c
    # ax.plot(x, y)
    return y

In [4]:
def plot_area_under_tangent_triange(z):

    x = np. linspace(0.1, 2.5, 100)
    y = f(x)
    
    
    fig, ax = plt.subplots()

    ax.plot(x,y)
    line_y = plot_line(f_der(z), tangent_y_intercept(z), x, ax=ax)
    ax.fill_between(x, line_y, where = line_y >= 0)
    ax.set_xlim(0.1, 2.5)
    ax.set_ylim(-1, 7)

    area = area_under_line(z)
    ax.annotate(f"area = {round(area)}", xy = (z, f(z)), arrowprops=dict(arrowstyle='->'), xytext=(1.5, 6))
    
interact(plot_area_under_tangent_triange, z = widgets.FloatSlider(value = 1 , min = 0.1, max = 2.5, step = 0.01))

interactive(children=(FloatSlider(value=1.0, description='z', max=2.5, min=0.1, step=0.01), Output()), _dom_cl…

<function __main__.plot_area_under_tangent_triange(z)>

### Example 2: $ x^n $

In [5]:
def f(x, n):
    return x ** n

In [6]:
def example_2(n):

    fig, ax = plt.subplots()
    x = np.linspace(-5,5,100)
    x_zero_mask = x==0
    y = f(x,n)

    # y[zero_index] == np.mean(y[zero_index-1], y[zero_index+1])
    
    ax.plot(x,y)

    ax.set_xlim(-5,5)
    ax.set_ylim(-5,5)
    
interact(example_2, n=widgets.FloatSlider(value = 1 , min = -5, max = 5, step = 1 ))

interactive(children=(FloatSlider(value=1.0, description='n', max=5.0, min=-5.0, step=1.0), Output()), _dom_cl…

<function __main__.example_2(n)>

I want to take the derivative for a given value of n. How can we do this? The binomial expansion of a nth degree polynomial will shows that:

$ (x + dx)^n = (x + dx)_1 * (x + dx)_2 * ... * (x + dx)_n  $

meaning we will always have 

## $ \frac{x^n +  nx^{n-1} + \omicron (\delta x)^2 - x^n}{\delta x} $

* that last part meaning we will will have multiple delta x terms that arent important

We will always have a $ -x^n $ term when we use the first priciples formula, and any terms with $ \delta x $ will divide out, therefore:

## $ \frac{\delta y}{\delta x} x^n = nx^{n-1} $

We can now easily implment a way of calculating a derivative.

In [7]:
class Polynomial:
    def __init__(self, a, n):
        self.a = a
        self.n = n

    def f(self, x):
        return self.a * (x ** self.n)

    def derive(self):
        return self.a * self.n, self.n - 1

In [8]:
poly1 = Polynomial(2, 2)
poly2 = Polynomial(*poly1.derive())

print(poly2.a, poly2.n)

4 1


In [9]:
def example_2_improved(a, n):
    
    poly1 = Polynomial(a, n)
    poly2 = Polynomial(*poly1.derive())
    
    fig, ax = plt.subplots()
    x = np.linspace(-5,5,100)
    x_zero_mask = x==0
    y1 = poly1.f(x)
    y2 = poly2.f(x) 

    # y[zero_index] == np.mean(y[zero_index-1], y[zero_index+1])
    
    ax.plot(x,y1)
    ax.plot(x,y2)

    ax.set_xlim(-5,5)
    ax.set_ylim(-5,5)
    
interact(
    example_2_improved,
    a=widgets.FloatSlider(value = 1 , min = -5, max = 5, step = 0.1 ),
    n=widgets.FloatSlider(value = 1 , min = -5, max = 5, step = 1 )
)

interactive(children=(FloatSlider(value=1.0, description='a', max=5.0, min=-5.0), FloatSlider(value=1.0, descr…

<function __main__.example_2_improved(a, n)>

This isnt't bad, but it would be nice if we could have multiple terms in the polynomial, such as:

$ f(x) = 2x^3 + 3x^2 + 1 $

In [10]:
class MultiPoly():
    def __init__(self, coefs: list):
        self.coefs = coefs
        
    def _f(self, x, coefs):
        sum = 0
        for exponent, coef in enumerate(coefs):
            sum += coef * (x ** exponent)
        
        return sum

    
    def f(self, x):
        return self._f(x, self.coefs)

    
    def _derive(self, coefs):
        
        new_coefs = []
        for i in range(1, len(coefs)):
            new_coefs.append(coefs[i] * i)
        
        return new_coefs

    def derive(self):
        return self._derive(self.coefs)


    def get_tangent_line_points(self, x1, z):

        # y = mx + c

        derivative_ceofs = self.derive()
        
        m = self._f(x1, derivative_ceofs)
        c = self.f(x1) - ( self._f(x1, derivative_ceofs) * x1 )

        return m * z + c
            
            

In [11]:
def line_m_c(p1, p2):
    x1, y1 = p1
    x2, y2 = p2

    dy = (y2 - y1) 
    dx = (x2 - x1)
    if dx == 0:
        # just moves the line out of the way
        return (0,100)
    
    m = dy / dx
    c = y1 - m * x1

    return(m,c)

In [12]:
def example_2_improved_2_secant_approx(x1,dx, a,b,c,d):

    fig, ax = plt.subplots()
    
    # polynomial instances
    poly1 = MultiPoly([a,b,c,d])
    poly2 = MultiPoly(poly1.derive())

    # plot points on poly1
    y1 = poly1.f(x1)
    x2 = x1 + dx
    y2 = poly1.f(x2)
    point_1 = (x1, y1) 
    point_2 = (x2, y2) 
    points = np.array([point_1, point_2]).T
    ax.scatter(points[0], points[1], marker='o')

    # plot poly1 and poly2
    x = np.linspace(-10,10,100)
    y1 = poly1.f(x)
    y2 = poly2.f(x)
    ax.plot(x,y1)
    # ax.plot(x,y2)

    # plot tangent to poly1
    tangent_x = np.linspace(-10,10, 10000)
    tangent_y = poly1.get_tangent_line_points(x1, tangent_x)
    ax.plot(tangent_x, tangent_y)

    # plot secant

    m,c = line_m_c(point_1,point_2)
    secant_y = m * x + c
    plt.plot(x,secant_y)
    
    ax.set_xlim(-10,10)
    ax.set_ylim(-10,10)
    
interact(
    example_2_improved_2_secant_approx,
    dx=widgets.FloatSlider(value=3, min=-6, max=5, step=0.05),
    x1=widgets.FloatSlider(value=1, min=-2.5, max=3, step=0.05),
    a=widgets.FloatSlider(value = 1 , min = -5, max = 5, step = 0.01 ),
    b=widgets.FloatSlider(value = 1 , min = -5, max = 5, step = 0.01 ),
    c=widgets.FloatSlider(value = 1 , min = -5, max = 5, step = 0.01 ),
    d=widgets.FloatSlider(value = 1 , min = -5, max = 5, step = 0.01 ),
)

interactive(children=(FloatSlider(value=1.0, description='x1', max=3.0, min=-2.5, step=0.05), FloatSlider(valu…

<function __main__.example_2_improved_2_secant_approx(x1, dx, a, b, c, d)>

## Example 3: Absolute value

$ f(x) = |x| $



In [13]:
def sigmoid(x,n):
    return (2 / (1+ np.pow(np.e,-x*n))) - 1

def abs_approx(n):
    fig, ax = plt.subplots()
    
    x = np.linspace(-5,5,1000)
    y = sigmoid(x,n)

    ax.plot(x,y)
    ax.plot(x,np.abs(x))
    
    ax.set_xlim(-5,5)
    ax.set_ylim(-2,2)

interact(abs_approx, n=widgets.FloatSlider(value = 1 , min = 0, max = 100, step = 1) )    

interactive(children=(FloatSlider(value=1.0, description='n', step=1.0), Output()), _dom_classes=('widget-inte…

<function __main__.abs_approx(n)>